## CS-470: Homework Set # 1
#### Name: GAO Xufeng. Sciper: 337088

In [1]:
import json
import numpy as np
import copy
import ctypes

**1. Parse Json: Parse JSON to get the program** 

In [2]:
def parseInstructions(file_name='test.json'):
    json_file = open(file_name)
    all_instructions = json.load(json_file)    
    return all_instructions

**2. Data Structure**

Three classes are defined:
- **Processor_state:** to store all data structues mentioned in the handout.
- **Integer_Queue_cell:** to indicate one entry of Integer Queue.
- **Active_List_cell:** to indicate one entry of Active List 

In [3]:
class Processor_state:
    
    # Function to initialize processor state
    def __init__(self):
        # 64 Physical Register File
        self.PhysicalRegisterFile = {}
        for i in range(64):
            self.PhysicalRegisterFile[i] = 0
    
        # 64 Busy Bit Table
        self.BusyBitTable = {}
        for i in range(64):
            self.BusyBitTable[i] = False
    
        # 32 Register Map Table
        self.RegisterMapTable = {}
        for i in range(32):
            self.RegisterMapTable[('x'+str(i))] = i
    
        # 32 Free List
        self.FreeList = [i for i in range(32,64)]
        # 32 Active List
        self.ActiveList = []
        # 32 Integer Queue
        self.IntegerQueue = []
        
        # Exception Mode Registers
        self.ExceptionRegister = {'ExceptionFlag': False, 'ExceptionPC': 0}
        # A wire going from the commit stage to the exception flag register
        self.bus_to_exception = {'ExceptionTrigger': False, 'ExceptionPC': None}
        
        # Decoded Instruction Register
        self.DIR = []
        self.DecodedPCs = []
        self.PC = 0
        self.Cycle = 0
        
        # Back-Pressure going from R&D to F&D
        self.backPressure = False
        
        # Pipeline register between Issue stage and Execution stage
        self.IssueQueue = []
        
        # Forwarding path to broadcast data
        self.forwarding_path = {}
        
        # Pipeline register between the 1st ALU cycle and 2nd ALU cycle
        self.ALU_registers = []
        
        # A renamed forwarding path to update the Active List (mimic the Commit Unit function)
        self.Done_or_Exception = {}
        
    # Functions to read these data structures 
    def get_PhysicalRegisterFile(self):
        return self.PhysicalRegisterFile
    def get_BusyBitTable(self):
        return self.BusyBitTable
    def get_RegisterMapTable(self):
        return self.RegisterMapTable
    def get_FreeList(self):
        return self.FreeList
    def get_ActiveList(self):
        return self.ActiveList
    def get_IntegerQueue(self):
        return self.IntegerQueue
    def get_ExceptionRegister(self):
        return self.ExceptionRegister
    def get_DIR(self):
        return self.DIR
    def get_DecodedPCs(self):
        return self.DecodedPCs
    def get_PC(self):
        return self.PC
    def get_Cycle(self):
        return self.Cycle
    def get_backPressure(self):
        return self.backPressure
    
    # Function to check those list/queue structures full
    def check_full(self, name):
        full_dict = {'DIR': 4==len(self.DIR), 'FreeList': 32==len(self.FreeList), \
                      'ActiveList': 32==len(self.ActiveList), 'IntegerQueue': 32==len(self.IntegerQueue), 'IssueQueue': 4==len(self.IssueQueue),
                     'ALURegisters': 4==len(self.ALU_registers), 'Done_or_Exception': 4==len(self.Done_or_Exception)}
        return full_dict[name]
    # Function to check those list/queue structures empty
    def check_empty(self, name):
        empty_dict = {'DIR': 0==len(self.DIR), 'FreeList': 0==len(self.FreeList), \
                      'ActiveList': 0==len(self.ActiveList), 'IntegerQueue': 0==len(self.IntegerQueue), 'IssueQueue': 0==len(self.IssueQueue),
                     'ALURegisters': 0==len(self.ALU_registers), 'Done_or_Exception': 0==len(self.Done_or_Exception)}
        return empty_dict[name]
    
    # Functions to display these list/queue structures
    def show_Active_List(self):
        return [cell.show_cell() for cell in self.ActiveList]
    def show_Integer_Queue(self):
        return [cell.show_cell() for cell in self.IntegerQueue]
    def show_Issue_Queue(self):
        return [cell.show_cell() for cell in self.IssueQueue]
    def show_ALU_Registers(self):
        return self.ALU_registers
    def show_Done_or_Exception(self):
        return self.Done_or_Exception
    
    
class Integer_Queue_cell:
    
    # Function to initialize integer queue entry
    def __init__(self):
        self.DestRegister = None
        self.OpAIsReady = False
        self.OpARegTag = None
        self.OpAValue = None
        self.OpBIsReady = False
        self.OpBRegTag = None
        self.OpBValue = None
        self.OpCode = None
        self.PC = None
        
    # Functions to assign entry parameters
    def set_fixed_args(self, DestRegister, OpCode, PC):
        self.DestRegister = DestRegister
        self.OpCode = OpCode
        self.PC = PC
    def set_OpA(self, OpIsReady, OpRegTag, OpValue):
        self.OpAIsReady = OpIsReady
        self.OpARegTag = OpRegTag
        self.OpAValue = OpValue
    def set_OpB(self, OpIsReady, OpRegTag, OpValue):
        self.OpBIsReady = OpIsReady
        self.OpBRegTag = OpRegTag
        self.OpBValue = OpValue
        
    # Function to display entry
    def show_cell(self):
        return {'DestRegister':self.DestRegister, 'OpAIsReady':self.OpAIsReady, 'OpARegTag':self.OpARegTag, 'OpAValue':self.OpAValue,\
                'OpBIsReady':self.OpBIsReady, 'OpBRegTag':self.OpBRegTag, 'OpBValue':self.OpBValue, 'OpCode':self.OpCode, 'PC':self.PC}
        
class Active_List_cell:
    
    # Function to initialize active list entry
    def __init__(self):
        self.Done = False
        self.Exception = False
        self.LogicalDest = None
        self.OldDest = None
        self.PC = None
    
    # Functions to assign entry parameters
    def set_Done(self, Done):
        self.Done = Done
    def set_Exception(self, Exception):
        self.Exception = Exception
    def set_fixed_args(self, PC, LogicalDest, OldDest):
        self.PC = PC
        self.LogicalDest = LogicalDest
        self.OldDest = OldDest
    
    # Function to display entry
    def show_cell(self):
        return {'Done':self.Done, 'Exception':self.Exception, 'LogicalDestination':self.LogicalDest, 'OldDestination':self.OldDest, 'PC':self.PC}
        


**3. Fetch & Decode Stage**

This stage keeps Fetching & Decoding instructions until there is an **Exception, BackPressure or No more instructions case**.

In [4]:
def Fetch_Decode_Stage(all_instructions, new_state):
    
    # if there are no exception and back-pressure, keep fetching and decoding
    if not new_state.get_ExceptionRegister()['ExceptionFlag']:
        if not new_state.get_backPressure():
            # check DIR and all_instruction
            while (not new_state.check_full('DIR') and len(all_instructions) != 0 and new_state.PC != hex(0x10000)):
                # Fetch instruction
                instruction = all_instructions.pop(0).replace(',', '').split(' ')
                DIR_cell = {'PC':new_state.PC, 'Ins':instruction}
                new_state.DIR.append(DIR_cell)
                
                # Decode the PC
                new_state.DecodedPCs.append(new_state.PC)
                # PC increament
                new_state.PC += 1
    else:
        # set the special PC value
        new_state.PC = hex(0x10000)
        # clear DIR
        new_state.DIR.clear()
        

**4. Rename & Dispatch Stage**

This stage keeps Renaming & Dispatching instructions until there is an **Exception, No more empty spaces in Active List/Integer Queue, No more free registers in Free List, No more DIR instructions case**.

In [5]:
def Rename_Dispatch_Stage(new_state):
    
    # if there are instructions in DIR and no exception
    while (not new_state.check_empty('DIR') and not new_state.get_ExceptionRegister()['ExceptionFlag']):
        # check Free list, Active List and Integer Queue
        if new_state.check_empty('FreeList') or new_state.check_full('ActiveList') or new_state.check_full('IntegerQueue'):
            # apply for backPressure
            new_state.backPressure = True
            break
        else:
            new_state.backPressure = False
            
            # fetch instruction from DIR
            DIR_cell = new_state.DIR.pop(0)
            DecodedPC_cell = new_state.DecodedPCs.pop(0)
            # take parameters
            DIR_PC = DIR_cell['PC']
            OpCode, DestArchiReg, OpA, OpB = DIR_cell['Ins']
           
            # determine state of operands
            OpARegTag, OpAValue, OpAIsReady = Operand_state(OpA, new_state)
            if OpCode == 'addi':
                OpBRegTag = 0
                OpBValue = int(OpB)
                OpBIsReady = True
                OpCode = 'add'
            elif OpCode in ['add', 'sub', 'mulu', 'divu', 'remu']:
                OpBRegTag, OpBValue, OpBIsReady = Operand_state(OpB, new_state)
            else:
                raise ValueError('Wrong OpCode')
        
            # update Free List & Register Map Table & the Busy bit
            new_DestPhyReg = new_state.FreeList.pop(0)
            Old_DestPhyReg = new_state.RegisterMapTable[DestArchiReg]
            new_state.RegisterMapTable[DestArchiReg] = new_DestPhyReg
            new_state.BusyBitTable[new_DestPhyReg] = True
                
            # update Active List
            Active_cell = Active_List_cell()
            LogicalDest = list(new_state.RegisterMapTable.keys()).index(DestArchiReg)
            Active_cell.set_fixed_args(DIR_PC, LogicalDest, Old_DestPhyReg)
            new_state.ActiveList.append(Active_cell)
            
            # update Integer Queue
            IQ_cell = Integer_Queue_cell()
            IQ_cell.set_fixed_args(new_DestPhyReg, OpCode, DIR_PC)
            IQ_cell.set_OpA(OpAIsReady, OpARegTag, OpAValue)
            IQ_cell.set_OpB(OpBIsReady, OpBRegTag, OpBValue)
            new_state.IntegerQueue.append(IQ_cell)
            
            
            
    # observe forwarding paths & update physical register file and busy bit table
    for RegTag in new_state.forwarding_path:
        new_state.PhysicalRegisterFile[RegTag] = new_state.forwarding_path[RegTag]
        new_state.BusyBitTable[RegTag] = False
    
# function to determine operand state
def Operand_state(Operand, new_state):

    OpRegTag = new_state.RegisterMapTable[Operand]
    if OpRegTag in new_state.forwarding_path:
        # from forwarding path
        OpValue = new_state.forwarding_path[OpRegTag]
        OpIsReady = True
    elif not new_state.BusyBitTable[OpRegTag]:
        # not busy, from physical register file
        OpValue = new_state.PhysicalRegisterFile[OpRegTag]
        OpIsReady = True
    else:
        # not produced yet
        OpValue = 0
        OpIsReady = False
    
    return OpRegTag, OpValue, OpIsReady
    

**5. Issue Stage**

This stage will issue 4 ready instructions (at most) until there is an **Exception or No more entries in Integer Queue case**. A pipeline register called **IssueQueue** (corresponding to register-3 in Fig.1 of handout) is used to store these issued instructions.

In [6]:
def Issue_Stage(new_state):
    
    # if there is exception, clear Integer Queue
    if new_state.get_ExceptionRegister()['ExceptionFlag']:
        new_state.IntegerQueue.clear()
        
    # check Integer Queue
    if not new_state.check_empty('IntegerQueue'):
        inds = []
        for ind, Issue_cell in enumerate(new_state.IntegerQueue):
            # observing forwarding path
            if (not Issue_cell.OpAIsReady) and (Issue_cell.OpARegTag in new_state.forwarding_path):
                Issue_cell.OpAValue = new_state.forwarding_path[Issue_cell.OpARegTag]
                Issue_cell.OpAIsReady = True
            # observing forwarding path
            if (not Issue_cell.OpBIsReady) and (Issue_cell.OpBRegTag in new_state.forwarding_path):
                Issue_cell.OpBValue = new_state.forwarding_path[Issue_cell.OpBRegTag]
                Issue_cell.OpBIsReady = True
        
            # check both operands ready
            if Issue_cell.OpAIsReady and Issue_cell.OpBIsReady:
                new_state.IssueQueue.append(Issue_cell)
                inds.append(ind)
                
            # check Issue Queue
            if new_state.check_full('IssueQueue'):
                break;
                
        # update Integer Queue (out of order)
        new_state.IntegerQueue = np.delete(new_state.IntegerQueue, inds).tolist()
                

**6. Execution Stage**

This stage takes **2 cycles**. 
- In the 1st cycle, the instructions (most 4) issued from previous stage are executed by the available ALUs. The calculated results will be stored in a pipeline register (corresponding to register-4 in Fig.1 of handout) called **ALU_registers** . 
- In the 2nd cycle, these results will be **broadcast** to forwarding paths.  
- This stage will be reset if there is an exception.
- Because this is a cycle-accurate simulator, the logics in 2nd ALU cycle will be executed before the ones in 1st cycle, updating each data structure correctly. 

In [7]:
def Execution_Stage(new_state):
    
    # if there is an exception
    if new_state.get_ExceptionRegister()['ExceptionFlag']:
        new_state.ALU_registers.clear()
        new_state.IssueQueue.clear()
    
    # we inverse the order of ALUs, so that they can be updatd correctly 
    # 2nd ALU cycle: update the forwarding path and Done_or_Exception path
    # before updating, we first clear the residual values (from past cycle) on both paths
    new_state.forwarding_path.clear()
    new_state.Done_or_Exception.clear()
    
    # check the calcualted results from 1st ALU cycle
    # Done_or_Exception is used to pass Done/Exception flags to Active List
    while (not new_state.check_empty('ALURegisters')) and (not new_state.check_full('Done_or_Exception')):
        # take result from ALU_registers
        ALU_cell = new_state.ALU_registers.pop(0)
        
        if ALU_cell['exception'] == False:
            # update forwarding path & Done_or_Exception path if exception is False
            new_state.forwarding_path[ALU_cell['DestRegister']] = ALU_cell['result']
            new_state.Done_or_Exception[ALU_cell['PC']] = 'Done'
        else:
            # update Done_or_Exception path if exception is True 
            new_state.Done_or_Exception[ALU_cell['PC']] = 'Exception'
    
    # 1st ALU cycle: perform issued instructions
    # here we iterate the IssueQueue at most 4 times to mimic 4 seperate ALUs
    while (not new_state.check_empty('IssueQueue')) and (not new_state.check_full('ALURegisters')):
        # take instruction from the Issue stage
        IssueQ_cell = new_state.IssueQueue.pop(0)
        # check if there is an exception
        if IssueQ_cell.OpCode in ['divu', 'remu'] and IssueQ_cell.OpBValue == 0:
            result = None
            exception = True
        else:
            result, exception = general_ALU(IssueQ_cell.OpCode, IssueQ_cell.OpAValue, IssueQ_cell.OpBValue)
        
        # store results on pipeline register, latch them into 2nd ALU stage at the next cycle
        new_state.ALU_registers.append({'exception':exception, 'DestRegister':IssueQ_cell.DestRegister, 'result':result, 'PC':IssueQ_cell.PC})
    

# function to convert int to unsigned 64 int
def unsigned(i):
    return ctypes.c_uint64(i).value

# ALU to perform different operations
def general_ALU(OpCode, OpA, OpB):
    if OpCode == 'add':
        result = OpA+OpB
    elif OpCode == 'sub':
        result = OpA-OpB
    elif OpCode == 'mulu':
        result = unsigned(OpA)*unsigned(OpB)
    elif OpCode == 'divu':
        result = int(unsigned(OpA)/unsigned(OpB))
    elif OpCode == 'remu':
        result = unsigned(OpA)%unsigned(OpB)

    return result, False


**7. Commit Stage**

This stage is considered to work with 2 modes:
1. **Normal mode (without exception):** **(1)** The active list is scanned  regularly, and at most 4 instructions will be retired in one cycle. **(2)** If the head instruction in active list is not completed yet, the commit stage will stall and only those instructions already picked will be retired. **(3)** When an instruction triggers an exception, the commit stage will stall. Because the Exception Mode works as an register in our system, the commit stage cannot change its content in current cycle. Thus, a special bus called **bus_to_exception** going from commit stage to exception register is introduced, which causes the contents of exception register to be updated in the next cycle. To make things easier, I assumed this bus include 2 fields: one for exception trigger signal, another for storing the exception PC. Thus, at the start of the next cycle, the **Exception_register** will set the exception flag and exception PC, and all other stages can read from it (i.e., can enter exception mode).
1. **Exception mode:** All other stages will see the exception flag and enter the exception mode, e.g., clear DIR. For commit stage, it will recover the left instructions from Active list bottom, and Register Map Table, Free List and Busy Bit Table will be recovered accordingly. Finally, when Active list become empty, the bus_to_exception will be reset in the same cycle. Thus, at the start of next cycle, all stages will know exception is handled, and they will quit the exception mode. In this last cycle, the PC is 0x10000, the Exception PC is the PC pointing to the instruction triggering exception, and the Exception flag is false.
1. **observe_Done_or_Exception_path** is used to mark entries of active list with DONE/EXCEPTION. The main reason for introducing this feature can be found in my submitted design handout.pdf. 

In [8]:
def Exception_register(new_state):
    # check if the bus_to_exception from Commit Unit (in previous cycle) is set
    if new_state.bus_to_exception['ExceptionTrigger'] == True:
        # set the excepton flag to True in current cycle
        # other units can see this True flag and enter Exception Mode in current cycle
        new_state.ExceptionRegister['ExceptionFlag'] = True
        
        # update the excepton PC
        new_state.ExceptionRegister['ExceptionPC'] = new_state.bus_to_exception['ExceptionPC']
    else:
        # clear exception flag
        new_state.ExceptionRegister['ExceptionFlag'] = False

def Commit_Unit(new_state):
    # If Exception Flag = True, enter the Exception Mode
    if new_state.ExceptionRegister['ExceptionFlag'] == True:
        # pick 4 instructions in reversed order
        recover_list = []
        while True:
            recover_cell = new_state.ActiveList.pop(-1)
            recover_list.append({'LogicalDest': 'x'+str(recover_cell.LogicalDest), 'Old_Dest':recover_cell.OldDest})
            # if Active list is empty
            if new_state.check_empty('ActiveList'):
                new_state.bus_to_exception['ExceptionTrigger'] = False
                break
            if len(recover_list) == 4:
                break
                
        # do the rolling back
        for re in recover_list:
            # recover Free List
            re_free_reg = new_state.RegisterMapTable[re['LogicalDest']]
            new_state.FreeList.insert(0, re_free_reg)
            
            # recover Register Map Table
            new_state.RegisterMapTable[re['LogicalDest']] = re['Old_Dest']
            
            # recover Busy bit table
            new_state.BusyBitTable[re_free_reg] = False
        
    else:
        # Normal Mode
        commit_OldDest = []
        for ind, ActiveList_cell in enumerate(new_state.ActiveList):
            if ActiveList_cell.Done == True and ActiveList_cell.Exception == False:
                commit_OldDest.append(ActiveList_cell.OldDest)
                if len(commit_OldDest) == 4:
                    break
                else:
                    continue
            elif ActiveList_cell.Done == False and ActiveList_cell.Exception == False:
                break
            elif ActiveList_cell.Done == False and ActiveList_cell.Exception == True:
        
                new_state.bus_to_exception['ExceptionTrigger'] = True
                new_state.bus_to_exception['ExceptionPC'] = ActiveList_cell.PC
    
        # retiring active list cells and freeing registers 
        for OldDest in commit_OldDest:
            new_state.ActiveList.pop(0)
            new_state.FreeList.append(OldDest)


def observe_Done_or_Exception_path(new_state):
    # observe Done_or_Exception paths & update active list
    for Active_list_cell in new_state.ActiveList:
        PCTag = Active_list_cell.PC
        if PCTag in new_state.Done_or_Exception:
            if new_state.Done_or_Exception[PCTag] == 'Done':
                Active_list_cell.Done = True
            if new_state.Done_or_Exception[PCTag] == 'Exception':
                Active_list_cell.Exception = True
                

**8. Propagate**

This function is used to prepare the next state of processor. The complete pipeline follows below order: **(1)** Commit_Stage = Exception_register + Commit_Unit + observe_Done_or_Exception_path (behind the Execution_Stage to meet the test_result); **(2)** Execution_Stage; **(3)** Issue_Stage; **(4)** Renamed_Dispatch_Stage; **(5)** Fetch_Decode_Stage. This kind of reverse order preparation will ensure all data structure are updated accordingly.


In [9]:
def propagate():
    
    # copy the original datasctructure
    new_state = copy.deepcopy(state)
    
    Exception_register(new_state)
    Commit_Unit(new_state)
    Execution_Stage(new_state)
    observe_Done_or_Exception_path(new_state)
    Issue_Stage(new_state)
    Rename_Dispatch_Stage(new_state)
    Fetch_Decode_Stage(all_instructions, new_state)
    
    return new_state


**8. Latch and saveLog**

Functions to update processor state, and save results in Json file.

In [10]:
def latch(update):
    state.__dict__.update(update.__dict__)
    state.Cycle += 1


def dumpStateIntoLog():
    print('---------------------------------')
    print('Cycle = ', state.get_Cycle())
    print('DIR list', state.get_DIR())
    print('---------------------------------')
    print('PC: ', state.get_PC())
    print('Decoded PCs', state.get_DecodedPCs())
    print('Under Exception', state.get_ExceptionRegister())
    print('Free List', state.get_FreeList())
    print('Physical register file', state.get_PhysicalRegisterFile())
    print('Busy bit Table', state.get_BusyBitTable())
    print('Architecture Register File', state.get_RegisterMapTable())
    
    AC_list = state.show_Active_List()
    print('Active list:')
    for ac in AC_list:
        print(ac)
        
    IQ = state.show_Integer_Queue()
    print('Integer Queue: ')
    for iq in IQ:
        print(iq)
      
    Issue_Q = state.show_Issue_Queue()
    print('Issue Queue: ')
    for Issue in Issue_Q:
        print(Issue)
        
    ALU_R = state.show_ALU_Registers()
    print('ALU Registers')
    for alu_r in ALU_R:
        print(alu_r)
    
    DOE = state.show_Done_or_Exception()
    print('Done or Exception', DOE)
        
    forward_p = state.forwarding_path
    print('Forwarding Path', forward_p)
    
    print('---------------------------------')

def write_test_results():
    current_cycle = {}
    current_cycle["ActiveList"] = state.show_Active_List()
    current_cycle["BusyBitTable"] = list(state.get_BusyBitTable().values())
    current_cycle["DecodedPCs"] = state.get_DecodedPCs()
    current_cycle["Exception"] = state.get_ExceptionRegister()['ExceptionFlag']
    current_cycle["ExceptionPC"] = state.get_ExceptionRegister()['ExceptionPC']
    current_cycle["FreeList"] = state.get_FreeList()
    current_cycle["IntegerQueue"] = state.show_Integer_Queue()
    current_cycle["PC"] = state.get_PC()
    current_cycle["PhysicalRegisterFile"] = list(state.get_PhysicalRegisterFile().values())
    current_cycle["RegisterMapTable"] = list(state.get_RegisterMapTable().values())
    test_result.append(current_cycle)
    
def saveLog(file_name='test_result.json'):
    # Serializing json 
    json_object = json.dumps([test for test in test_result], indent = 4)

    # Writing to sample.json
    with open(file_name, "w") as outfile:
        outfile.write(json_object)

**9. Main Program**

The main program will finish when **No more instructions, No more Active list entries and Exception mode is exit if there is an exception**.

In [11]:
# input JSON file
all_instructions = parseInstructions(file_name='test.json')
state = Processor_state()
#dumpStateIntoLog()
test_result = []
write_test_results()

while not (len(all_instructions) == 0 and len(state.get_DecodedPCs()) == 0 and state.check_empty('ActiveList') and state.ExceptionRegister['ExceptionFlag'] == False):
    update = propagate()
    latch(update)
    #dumpStateIntoLog()
    write_test_results()
    
# output JSON file
saveLog(file_name='test_result.json')